# Imports

In [1]:
from math import ceil
from time import sleep
from random import randint
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import requests
import json


# Scraping

## Config

In [2]:
base_url = 'https://www.kisskissbankbank.com'

## Getting URLs


In [3]:
url = f'{base_url}/fr/discover?project[successful]=on&filter=all'

response = requests.get(url)
soup = BeautifulSoup(response.content, "html5lib")

script = json.loads(soup.find('script', class_='js-react-on-rails-component').text)
nb_projects = script['BROWSING']['projects']['totalCount']

nb_pages = ceil(nb_projects/9)

In [5]:
links = []
browser = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(browser, 100)

for page_number in range(1, nb_pages+1):
    url = f'{base_url}/fr/discover?filter=all&project[successful]=on&page={page_number}'
    browser.get(url)
    sleep(2)
    
    for item in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.k-LegoGrid__item__content>a'))): 
        cats = item.find_element_by_css_selector('div.k-CrowdfundingCard__subtitle__container>p>span>span>span').text
        links.append({'url': item.get_attribute('href'), 'cats': cats})
    
    if page_number % 100 == 0:
        print(f'Page {page_number} / {nb_pages+1} treated')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\tangu\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache


 
Page 100 / 2491 treated
Page 200 / 2491 treated
Page 300 / 2491 treated
Page 400 / 2491 treated
Page 500 / 2491 treated
Page 600 / 2491 treated
Page 700 / 2491 treated
Page 800 / 2491 treated
Page 900 / 2491 treated
Page 1000 / 2491 treated
Page 1100 / 2491 treated
Page 1200 / 2491 treated
Page 1300 / 2491 treated
Page 1400 / 2491 treated
Page 1500 / 2491 treated
Page 1600 / 2491 treated
Page 1700 / 2491 treated
Page 1800 / 2491 treated
Page 1900 / 2491 treated
Page 2000 / 2491 treated
Page 2100 / 2491 treated
Page 2200 / 2491 treated
Page 2300 / 2491 treated
Page 2400 / 2491 treated


In [11]:
pd.DataFrame(links).drop_duplicates('url', keep='first').to_pickle('Data/links.pkl')

In [7]:
df = pd.DataFrame(links)
df.shape

(22410, 2)

In [8]:
len(list(df.url.unique()))

21816